# OpenAI vectorizeモジュールを使用したWeaviateでのハイブリッド検索

このノートブックは以下のシナリオ向けに準備されています：
* データがベクトル化されていない
* データに対してハイブリッド検索（[詳細はこちら](https://weaviate.io/blog/hybrid-search-explained)）を実行したい
* OpenAIモジュール（[text2vec-openai](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-openai)）を使用してWeaviateでベクトル埋め込みを生成したい

このノートブックでは、Weaviateインスタンスのセットアップ、接続（OpenAI APIキーを使用）、データスキーマの設定、データのインポート（データのベクトル埋め込みを自動生成）、ハイブリッド検索（ベクトル検索とBM25検索の組み合わせ）の実行という簡単なフローを説明します。

これは、チャットボット、トピックモデリングなどの本番ユースケースをサポートするために、セキュアな環境で独自のデータと埋め込みを保存・検索したい顧客の一般的な要件です。

## Weaviateとは

Weaviateは、データオブジェクトをそのベクトルと一緒に保存するオープンソースのベクトル検索エンジンです。これにより、ベクトル検索と構造化フィルタリングを組み合わせることができます。

WeaviateはKNNアルゴリズムを使用してベクトル最適化されたインデックスを作成し、クエリを極めて高速に実行できます。詳細は[こちら](https://weaviate.io/blog/why-is-vector-search-so-fast)をご覧ください。

Weaviateでは、お気に入りのMLモデルを使用でき、数十億のデータオブジェクトまでシームレスにスケールできます。

### デプロイメントオプション

シナリオや本番セットアップに関係なく、Weaviateには適切なオプションがあります。以下のセットアップでWeaviateをデプロイできます：
* セルフホスト – dockerを使用してローカルまたは任意のサーバーにWeaviateをデプロイできます
* SaaS – [Weaviate Cloud Service (WCS)](https://console.weaviate.io/)を使用してWeaviateインスタンスをホストできます
* ハイブリッドSaaS – 独自のプライベートクラウドサービスにWeaviateをデプロイできます

### プログラミング言語

Weaviateは4つの[クライアントライブラリ](https://weaviate.io/developers/weaviate/client-libraries)を提供しており、アプリケーションから通信できます：
* [Python](https://weaviate.io/developers/weaviate/client-libraries/python)
* [JavaScript](https://weaviate.io/developers/weaviate/client-libraries/javascript)
* [Java](https://weaviate.io/developers/weaviate/client-libraries/java)
* [Go](https://weaviate.io/developers/weaviate/client-libraries/go)

さらに、Weaviateには[RESTレイヤー](https://weaviate.io/developers/weaviate/api/rest/objects)があります。基本的に、RESTリクエストをサポートする任意の言語からWeaviateを呼び出すことができます。

## デモフロー
デモフローは以下の通りです：
- **前提条件のセットアップ**: Weaviateインスタンスを作成し、必要なライブラリをインストール
- **接続**: Weaviateインスタンスに接続
- **スキーマ設定**: データのスキーマを設定
    - *注意*: ここで使用するOpenAI Embedding Modelを定義できます
    - *注意*: ここでインデックス化するプロパティを設定できます
- **データのインポート**: デモデータセットを読み込み、Weaviateにインポート
    - *注意*: インポートプロセスは、スキーマの設定に基づいてデータを自動的にインデックス化します
    - *注意*: データを明示的にベクトル化する必要はありません。WeaviateがOpenAIと通信して処理を行います
- **クエリの実行**: クエリを実行
    - *注意*: クエリを明示的にベクトル化する必要はありません。WeaviateがOpenAIと通信して処理を行います

このノートブックを一通り実行すると、ベクトルデータベースのセットアップと使用方法の基本的な理解が得られ、私たちのembeddingsを活用したより複雑なユースケースに進むことができます。

## WeaviateのOpenAIモジュール
すべてのWeaviateインスタンスには[text2vec-openai](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-openai)モジュールが搭載されています。

このモジュールは、インポート時（またはあらゆるCRUD操作時）およびクエリ実行時のベクトル化処理を担当します。

### データを手動でベクトル化する必要がありません
これはあなたにとって朗報です。[text2vec-openai](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-openai)を使用すれば、Weaviateが必要に応じてOpenAIを呼び出してくれるため、データを手動でベクトル化する必要がありません。

必要なのは以下だけです：
1. OpenAI API Keyを提供する – Weaviate Clientに接続する際
2. スキーマで使用するOpenAIベクトライザーを定義する

## 前提条件

このプロジェクトを開始する前に、以下の設定が必要です：

* `Weaviate`インスタンスの作成
* ライブラリのインストール
    * `weaviate-client`
    * `datasets`
    * `apache-beam`
* [OpenAI API key](https://beta.openai.com/account/api-keys)の取得

===========================================================
### Weaviateインスタンスの作成

Weaviateインスタンスを作成するには2つのオプションがあります：

1. （推奨方法）[Weaviate Cloud Service](https://console.weaviate.io/) – クラウドでWeaviateインスタンスをホストします。無料のサンドボックスでこのクックブックには十分です。
2. DockerでWeaviateをローカルにインストールして実行します。

#### オプション1 – WCSインストール手順

[Weaviate Cloud Service](https://console.weaviate.io/)（WCS）を使用して無料のWeaviateクラスターを作成します。
1. [WCS](https://console.weaviate.io/)で無料アカウントを作成および/またはログインします
2. 以下の設定で`Weaviate Cluster`を作成します：
    * Sandbox: `Sandbox Free`
    * Weaviate Version: デフォルトを使用（最新版）
    * OIDC Authentication: `Disabled`
3. インスタンスは1〜2分で準備完了します
4. `Cluster Id`をメモしてください。リンクをクリックするとクラスターの完全なパス（後で接続するために必要）が表示されます。`https://your-project-name.weaviate.network`のような形式になります

#### オプション2 – DockerでローカルWeaviateインスタンス

DockerでWeaviateをローカルにインストールして実行します。
1. [./docker-compose.yml](./docker-compose.yml)ファイルをダウンロードします
2. ターミナルを開き、docker-compose.ymlファイルがある場所に移動して、`docker-compose up -d`でDockerを起動します
3. 準備が完了すると、インスタンスは[http://localhost:8080](http://localhost:8080)で利用可能になります

注意：Dockerインスタンスをシャットダウンするには`docker-compose down`を実行してください

##### 詳細情報
DockerでWeaviateを使用する詳細については、[インストールドキュメント](https://weaviate.io/developers/weaviate/installation/docker-compose)を参照してください。

===========================================================    
## 必要なライブラリのインストール

このプロジェクトを実行する前に、以下のライブラリが必要です：

### Weaviate Python client

[Weaviate Python client](https://weaviate.io/developers/weaviate/client-libraries/python)を使用すると、PythonプロジェクトからWeaviateインスタンスと通信できます。

### datasets & apache-beam

サンプルデータを読み込むには、`datasets`ライブラリとその依存関係である`apache-beam`が必要です。

In [ ]:
# Install the Weaviate client for Python
!pip install weaviate-client>3.11.0

# Install datasets and apache-beam to load the sample datasets
!pip install datasets apache-beam

## OpenAI API キーの準備

`OpenAI API key`は、インポート時のデータのベクトル化と、クエリの実行に使用されます。

OpenAI API キーをお持ちでない場合は、[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

キーを取得したら、環境変数として`OPENAI_API_KEY`に追加してください。

In [ ]:
# Export OpenAI API Key
!export OPENAI_API_KEY="your key"

In [ ]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
# os.environ['OPENAI_API_KEY'] = 'your-key-goes-here'

if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

## Weaviateインスタンスに接続する

このセクションでは、以下を行います：

1. 環境変数 `OPENAI_API_KEY` をテストします – [#Prepare-your-OpenAI-API-key](#Prepare-your-OpenAI-API-key) のステップを完了していることを**確認してください**
2. あなたの `OpenAI API Key` を使用してWeaviateに接続します
3. クライアント接続をテストします

### クライアント

このステップの後、`client` オブジェクトはすべてのWeaviate関連の操作を実行するために使用されます。

In [ ]:
import weaviate
from datasets import load_dataset
import os

# Connect to your Weaviate instance
client = weaviate.Client(
    url="https://your-wcs-instance-name.weaviate.network/",
#   url="http://localhost:8080/",
    auth_client_secret=weaviate.auth.AuthApiKey(api_key="<YOUR-WEAVIATE-API-KEY>"), # comment out this line if you are not using authentication for your Weaviate instance (i.e. for locally deployed instances)
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")
    }
)

# Check if your instance is live and ready
# This should return `True`
client.is_ready()

# スキーマ

このセクションでは、以下を行います：
1. データのデータスキーマを設定する
2. OpenAIモジュールを選択する

> これは2番目で最後のステップであり、OpenAI固有の設定が必要です。
> このステップの後、残りの手順はWeaviateのみに関わるものとなります。OpenAIのタスクは自動的に処理されるためです。


## スキーマとは

Weaviateでは、検索対象となる各エンティティを捉えるために__スキーマ__を作成します。

スキーマは、Weaviateに以下を伝える方法です：
* データをベクトル化するために使用すべき埋め込みモデル
* データの構成要素（プロパティ名と型）
* どのプロパティをベクトル化してインデックス化すべきか

このクックブックでは、以下を含む`Articles`のデータセットを使用します：
* `title`
* `content`
* `url`

`title`と`content`をベクトル化したいのですが、`url`はベクトル化しません。

データをベクトル化してクエリするために、`text-embedding-3-small`を使用します。

In [ ]:
# Clear up the schema, so that we can recreate it
client.schema.delete_all()
client.schema.get()

# Define the Schema object to use `text-embedding-3-small` on `title` and `content`, but skip it for `url`
article_schema = {
    "class": "Article",
    "description": "A collection of articles",
    "vectorizer": "text2vec-openai",
    "moduleConfig": {
        "text2vec-openai": {
          "model": "ada",
          "modelVersion": "002",
          "type": "text"
        }
    },
    "properties": [{
        "name": "title",
        "description": "Title of the article",
        "dataType": ["string"]
    },
    {
        "name": "content",
        "description": "Contents of the article",
        "dataType": ["text"]
    },
    {
        "name": "url",
        "description": "URL to the article",
        "dataType": ["string"],
        "moduleConfig": { "text2vec-openai": { "skip": True } }
    }]
}

# add the Article schema
client.schema.create_class(article_schema)

# get the schema to make sure it worked
client.schema.get()

## データのインポート

このセクションでは以下を行います：
1. Simple Wikipediaデータセットを読み込む
2. Weaviate Batchインポートを設定する（インポートをより効率的にするため）
3. データをWeaviateにインポートする

> 注意: <br/>
> 前述したように、データを手動でベクトル化する必要はありません。<br/>
> [text2vec-openai](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-openai)モジュールがそれを処理します。

In [ ]:
### STEP 1 - load the dataset

from datasets import load_dataset
from typing import List, Iterator

# We'll use the datasets library to pull the Simple Wikipedia dataset for embedding
dataset = list(load_dataset("wikipedia", "20220301.simple")["train"])

# For testing, limited to 2.5k articles for demo purposes
dataset = dataset[:2_500]

# Limited to 25k articles for larger demo purposes
# dataset = dataset[:25_000]

# for free OpenAI acounts, you can use 50 objects
# dataset = dataset[:50]

In [ ]:
### Step 2 - configure Weaviate Batch, with
# - starting batch size of 100
# - dynamically increase/decrease based on performance
# - add timeout retries if something goes wrong

client.batch.configure(
    batch_size=10, 
    dynamic=True,
    timeout_retries=3,
#   callback=None,
)

In [ ]:
### Step 3 - import data

print("Importing Articles")

counter=0

with client.batch as batch:
    for article in dataset:
        if (counter %10 == 0):
            print(f"Import {counter} / {len(dataset)} ")

        properties = {
            "title": article["title"],
            "content": article["text"],
            "url": article["url"]
        }
        
        batch.add_data_object(properties, "Article")
        counter = counter+1

print("Importing Articles complete")       

In [ ]:
# Test that all data has loaded – get object count
result = (
    client.query.aggregate("Article")
    .with_fields("meta { count }")
    .do()
)
print("Object count: ", result["data"]["Aggregate"]["Article"], "\n")

In [ ]:
# Test one article has worked by checking one object
test_article = (
    client.query
    .get("Article", ["title", "url", "content"])
    .with_limit(1)
    .do()
)["data"]["Get"]["Article"][0]

print(test_article['title'])
print(test_article['url'])
print(test_article['content'])

### データ検索

上記のように、新しいIndexに対していくつかのクエリを実行し、既存のベクトルとの近似度に基づいて結果を取得します。

`alpha`設定について詳しくは[こちら](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#hybrid)をご覧ください。

In [ ]:
def hybrid_query_weaviate(query, collection_name, alpha_val):
    
    nearText = {
        "concepts": [query],
        "distance": 0.7,
    }

    properties = [
        "title", "content", "url",
        "_additional { score }"
    ]

    result = (
        client.query
        .get(collection_name, properties)
        .with_hybrid(nearText, alpha=alpha_val)
        .with_limit(10)
        .do()
    )
    
    # Check for errors
    if ("errors" in result):
        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")
        raise Exception(result["errors"][0]['message'])
    
    return result["data"]["Get"][collection_name]

In [ ]:
query_result = hybrid_query_weaviate("modern art in Europe", "Article", 0.5)

for i, article in enumerate(query_result):
    print(f"{i+1}. { article['title']} (Score: {article['_additional']['score']})")

In [ ]:
query_result = hybrid_query_weaviate("Famous battles in Scottish history", "Article", 0.5)

for i, article in enumerate(query_result):
    print(f"{i+1}. { article['title']} (Score: {article['_additional']['score']})")

フォローしていただき、ありがとうございました。これで独自のベクターデータベースを設定し、埋め込みを使用してあらゆる種類のクールなことを行う準備が整いました - お楽しみください！より複雑なユースケースについては、このリポジトリの他のクックブックの例を引き続き参照してください。